<a href="https://colab.research.google.com/github/DivyanshuBITS/Machine-Learning-Project/blob/main/Superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv('superstore.csv', encoding='latin-1')

**Exploratory Data Analysis**

In [ ]:
df.head()

In [ ]:
df.info()

There is no null value in dataset.

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df['Order Date'].unique().shape[0]

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
orders_by_month_year = df.groupby([df['Order Date'].dt.year, df['Order Date'].dt.month]).size().unstack(fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(orders_by_month_year, cmap='viridis', annot=True, fmt='d')
plt.xlabel('Month')
plt.ylabel('Year')
plt.title('Order Counts by Year and Month')
plt.show()

* The last four months of the year (September to December) are consistently the busiest period.
* Consistent year over-year growth


In [ ]:
#now let's see if there are missing values
missing = df.isnull().sum()
print(missing)

* There is no missing value.

In [ ]:
df.describe()

* The maximum sale is very high ($22,638),  compared to average $229, which strongly indicates the presence of outliers or a few very large orders.
* The minimum profit is highly negative ($-6,599). This is showing that some sales are extremely unprofitable.

In [ ]:
#columns has unique values with their frequencies
categorical_cols = ['Segment', 'Category', 'Sub-Category', 'Region', 'State', 'Ship Mode']
for col in categorical_cols:
    if col in df.columns:
        print(f"\n{col} :-")
        print(f"Unique values: {df[col].nunique()}")
        print(df[col].value_counts())

* Office Supplies is the most sold category by volume.
* Binders and Paper are the most frequently sold individual product.

In [ ]:
df['Ship Date']

*ship date is not in datetime.

In [ ]:
df['Order Date']

* Since Order date is in datetime object there is no need for any conversion







# **Network Analysis of Product Relationships**

*Gonna see which products are frequently bought together.

In [ ]:
#preparing data for network analysis
#grouping by order id to get all product names in each order
order_products = df.groupby('Order ID')['Product Name'].apply(list).reset_index()

In [ ]:
# filtering out orders with only one product as they don't form pairs
order_products = order_products[order_products['Product Name'].apply(len) > 1]

In [ ]:
from itertools import combinations

In [ ]:
# Create edges (pairs of products) for each order using itertools.combinations
edges = []
for products_list in order_products['Product Name']:
    # Sorting products in the list to ensure consistent pair order (for example-('A', 'B') is same as ('B', 'A'))
    sorted_products = sorted(products_list)
    for pair in combinations(sorted_products, 2):
        edges.append(pair)

In [ ]:
# now let's count the frequency of each product pair
edge_counts = pd.Series(edges).value_counts().reset_index()
edge_counts.columns = ['Product Pair', 'Frequency']

In [ ]:
# Displaying the top most frequent product pairs for network visualization
top_n_pairs = 25
most_frequent_pairs = edge_counts.head(25)
print(most_frequent_pairs)

Observation - so there are 24 product pairs exists.

In [ ]:
import networkx as nx

In [ ]:
#building the network graph
G = nx.Graph() #this will create an empty undirected graph

In [ ]:
#adding nodes and edges, here nodes are individual products and edges are co-purchase relationship
for index, row in most_frequent_pairs.iterrows():
    product1, product2 = row['Product Pair']
    frequency = row['Frequency']
    G.add_edge(product1, product2, weight = frequency)

# Visualization of the Network Graph

In [ ]:
plt.figure(figsize = (20,16)) # I have increased figure size for better visualization
pos = nx.spring_layout(G, k = 0.9, iterations = 70, seed = 42)
# nodes (products)
nx.draw_networkx_nodes(G, pos, node_size=4500, node_color='lightseagreen', alpha=0.9, linewidths=1.5, edgecolors='darkslategray')
# edges(co-purchase links)
max_freq = most_frequent_pairs['Frequency'].max()
edge_widths = [d['weight'] / max_freq * 10 for u, v, d in G.edges(data=True)]
nx.draw_networkx_edges(G, pos, edge_color='gray', width=edge_widths, alpha=0.6)
nx.draw_networkx_labels(G, pos, font_size=5, font_color='black')
plt.title(f"Network Analysis of Products Co-Purchases")
plt.axis('off')
plt.show()


* nodes that are very close together are highly connected to each other.
In the spring_layout algorithm's simulation, the "springs" (representing edges/connections) between these nodes are very strong because the products they represent are very frequently co-purchased. These strong attractive forces pull them into tight clusters.These are natural product groupings or bundles that customers consistently buy together.

* This technique leverages graph theory to visually represent product co-purchase patterns. Instead of just showing lists of frequently bought items, it constructs a network where each 'node' is a product and a connection(edge) signifies co-purchase within an order. The thickness of the edge directly indicates the frequency and strength of this relationship.

# **Time-Series Clustering of Customer Behavior**

In [ ]:
#here my goal is to identify distinct customer segments based on their monthly purchasing patterns over time.
#now i know that order date is a datetime object
df['Order Month'] = df['Order Date'].dt.to_period('M')
#now i'm grouping the customer id and order month to get monthly sales for each customer
customer_monthly_sales = df.groupby(['Customer ID', 'Order Month'])['Sales'].sum().unstack(fill_value=0)
all_months = pd.period_range(start = df['Order Month'].min(), end = df['Order Month'].max(), freq = 'M') #ensuring all customers have time series of same length , M is for monthly frequency
customer_monthly_sales = customer_monthly_sales.reindex(columns=all_months, fill_value=0)

In [ ]:
#Data Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #standard scaler ensures mean 0 and standard deviation 1.
customer_monthly_sales_scaled = scaler.fit_transform(customer_monthly_sales)
customer_monthly_sales_scaled = pd.DataFrame(customer_monthly_sales_scaled,index = customer_monthly_sales.index, columns=customer_monthly_sales.columns)

In [ ]:
print(customer_monthly_sales_scaled.iloc[:5, :5])

* negative values are indicating that the customer's sales for a given month is less than the average sakes for that month across all customers.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
#clustering
n = 5
cluster = AgglomerativeClustering(n_clusters=n)
#customer_monthly_sales_scaled.columns = customer_monthly_sales_scaled.columns.astype(str)
customer_monthly_sales['Cluster'] = cluster.fit_predict(customer_monthly_sales_scaled.values)

In [ ]:
customer_monthly_sales['Cluster'].value_counts()

In [ ]:
cluster_profiles = customer_monthly_sales.groupby('Cluster').mean()

In [ ]:
cluster_profiles.columns

In [ ]:
cluster_profiles.head()

In [ ]:
plt.figure(figsize=(18,10))
#transposing the dataframe so that months are on the x axis and each column represent a cluster's average sales
cluster_profiles_T = cluster_profiles.T

In [ ]:
# Plot each cluster's average monthly sales over time
plt.figure(figsize=(18, 10))
for cluster_id in cluster_profiles_T.columns:
    x_values = pd.to_datetime(cluster_profiles_T.index.astype(str))
    y_values = cluster_profiles_T[cluster_id]
    plt.plot(x_values, y_values, label=f'Cluster {cluster_id}')

plt.title('Time Series Clustering of Customer Behavior', fontsize=20, weight='bold')
plt.xlabel('Month', fontsize=14)
plt.ylabel('Average Monthly Sales', fontsize=14)
plt.legend(title='Customer Cluster')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# Geospatial Analysis

* now creating a choropleth map of the United States to visualize sales and profitability by state. This is far more impactful than a simple bar chart, as it instantly reveals regional strengths, weaknesses, and potential market opportunities.

In [ ]:
import plotly.express as px
#let's prepare data by states
df['State'] = df['State'].str.strip()#removing leading or trailing whitespaces from state names
state_data = df.groupby('State').agg(Total_sales = ('Sales', 'sum'), Total_profit = ('Profit', 'sum'), Customer_Count = ('Customer ID', 'nunique')).reset_index()
#profit  ratio
state_data['Profit_Ratio'] = (state_data['Total_profit'] / state_data['Total_sales'])*100
state_data.head()

In [ ]:
df['State']

In [ ]:
# #now creating the interactive map and coloring the states based on their total sales
# fig = px.choropleth(state_data,
#                     locations = 'State',
#                     locationmode = 'USA-states',
#                     color = 'Total_sales',
#                     scope = 'usa',
#                     hover_name='State',
#                     hover_data={'Total_profit': ':.2f', 'Profit_Ratio': ':.2f', 'Customer_Count': True },
#                     color_continuous_scale="Viridis",
#                     title="Geospatial Analysis: Total Sales & Profitability by State" )
# fig.show()

In [ ]:
state_data.info()

# Recency, Frequency, and Monetary Calculation

* recency is how recently a customer has made a purchase. Low recency value(higher recency score) shows recent purchase.

In [ ]:
# snapshot date for recency calculation (one day after the last order)
snapshot_date = df['Order Date'].max() + pd.Timedelta(days=1)
rfm_data = df.groupby('Customer ID').agg({'Order Date': lambda date: (snapshot_date - date.max()).days,'Order ID': 'nunique', 'Sales': 'sum'})
rfm_data.rename(columns={'Order Date': 'Recency','Order ID': 'Frequency','Sales': 'MonetaryValue'}, inplace=True)
#Lower Recency is better, Higher Frequency and Monetary are better.
r_labels = range(5, 0, -1)
f_labels = range(1, 6)
m_labels = range(1, 6)
rfm_data['R_Score'] = pd.qcut(rfm_data['Recency'], 5, labels=r_labels, duplicates='drop').astype(int)
rfm_data['F_Score'] = pd.qcut(rfm_data['Frequency'], 5, labels=f_labels, duplicates='drop').astype(int)
rfm_data['M_Score'] = pd.qcut(rfm_data['MonetaryValue'], 5, labels=m_labels, duplicates='drop').astype(int)
#using the weighted formula: 50% Recency, 25% Frequency, 25% Monetary
rfm_data['Customer_Health_Score'] = (0.5 * rfm_data['R_Score']) + (0.25 * rfm_data['F_Score']) + (0.25 * rfm_data['M_Score'])
# Normalize the score to a 0-100
max_possible_score = (0.5*5) + (0.25*5) + (0.25*5) # Max score is 5
min_possible_score = (0.5*1) + (0.25*1) + (0.25*1) # Min score is 1
rfm_data['Customer_Health_Score'] = ((rfm_data['Customer_Health_Score'] - min_possible_score) / (max_possible_score - min_possible_score)) * 100
#Display the top customers by health score
print(rfm_data.sort_values('Customer_Health_Score', ascending=False).head())

In [ ]:
#Aggregate data by product
product_data = df.groupby('Product Name').agg(Total_Sales=('Sales', 'sum'),Total_Profit=('Profit', 'sum')).reset_index()
# Filter out products with zero or negative sales to avoid division by zero
product_data = product_data[product_data['Total_Sales'] > 0]
product_data['Profit_Margin'] = (product_data['Total_Profit'] / product_data['Total_Sales']) * 100
# Rank products by sales and profit margin
product_data['Sales_Rank_Score'] = product_data['Total_Sales'].rank(pct=True)
product_data['Margin_Rank_Score'] = product_data['Profit_Margin'].rank(pct=True)
# We give equal weight to sales volume and profitability
product_data['Product_Velocity_Index'] = (product_data['Sales_Rank_Score'] + product_data['Margin_Rank_Score']) * 50 # Scale to 100
# Display the top products by velocity index
print(product_data.sort_values('Product_Velocity_Index', ascending=False).head())

* now calculating the number of unique customers in each state to identify regions with a strong customer base.

In [ ]:
#Calculate unique customers per state
market_penetration = df.groupby('State')['Customer ID'].nunique().reset_index()
market_penetration.rename(columns={'Customer ID': 'Unique_Customer_Count'}, inplace=True)
market_penetration = market_penetration.sort_values('Unique_Customer_Count', ascending=False)
plt.figure(figsize=(15, 10))
sns.barplot(x='Unique_Customer_Count', y='State', data=market_penetration, palette='coolwarm')
plt.title('Market Penetration: Unique Customers per State', fontsize=16)
plt.xlabel('Number of Unique Customers', fontsize=12)
plt.ylabel('State', fontsize=12)
plt.show()
print("--- Top 5 States by Customer Count ---")
print(market_penetration.head())
print("\n--- Bottom 5 States by Customer Count ---")
print(market_penetration.tail())

In [ ]:
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
# Defining promised shipping times
promised_times = {'Same Day': 0,'First Class': 1,'Second Class': 3,'Standard Class': 5}
df['Promised_Time'] = df['Ship Mode'].map(promised_times)
# Calculating actual shipping time
df['Actual_Ship_Time'] = (df['Ship Date'] - df['Order Date']).dt.days
# checking if each order was on time -
df['Is_On_Time'] = df['Actual_Ship_Time'] <= df['Promised_Time']

on_time_percentage = df['Is_On_Time'].mean() * 100
print(f"--- Operational Excellence Score ---")
print(f"On-Time Shipping Percentage: {on_time_percentage:.2f}%")

In [ ]:
# Aggregate data by Category
category_data = df.groupby('Category').agg( Total_Sales=('Sales', 'sum'), Total_Profit=('Profit', 'sum')).reset_index()

# Calculate Market Share and Profit Margin
total_company_sales = category_data['Total_Sales'].sum()
category_data['Market_Share'] = (category_data['Total_Sales'] / total_company_sales)
category_data['Profit_Margin'] = (category_data['Total_Profit'] / category_data['Total_Sales'])
category_data['Market_Share_Score'] = category_data['Market_Share'].rank(pct=True)
category_data['Profit_Margin_Score'] = category_data['Profit_Margin'].rank(pct=True)
category_data['Competitive_Advantage_Index'] = (category_data['Market_Share_Score'] + category_data['Profit_Margin_Score']) * 50
print(category_data.sort_values('Competitive_Advantage_Index', ascending=False))

# **Survival Analysis For Customer Churn**

In [ ]:
pip install lifelines

In [ ]:
from lifelines import CoxPHFitter

In [ ]:
#Feature Engineering
customer_features = df.groupby('Customer ID').agg(
    duration=('Order Date', lambda date: (date.max() - date.min()).days),
    Total_Sales=('Sales', 'sum'),
    Avg_Discount=('Discount', 'mean'),
    Total_Orders=('Order ID', 'nunique')
).reset_index()

In [ ]:
#now let's define churn event by a common business rule: a customer is considered "churned" if their last purchase was more than 6 months ago.
observation_end_date = df['Order Date'].max()
churn_cutoff_date = observation_end_date - pd.DateOffset(months=6)
# the last purchase date for each customer
last_purchase_dates = df.groupby('Customer ID')['Order Date'].max().reset_index()
last_purchase_dates.rename(columns={'Order Date': 'Last_Purchase_Date'}, inplace=True)
customer_features = pd.merge(customer_features, last_purchase_dates, on='Customer ID')
# The 'churned' event is 1 if their last purchase was before the cutoff, else 0.
customer_features['churned'] = (customer_features['Last_Purchase_Date'] < churn_cutoff_date).astype(int)

In [ ]:
# Cox Proportional Hazards Model - This model will tell us the effect of each feature on the likelihood of churn.
cph = CoxPHFitter()
cph.fit(customer_features[['duration', 'churned', 'Total_Sales', 'Avg_Discount', 'Total_Orders']], duration_col='duration', event_col='churned')

In [ ]:
cph.print_summary()

# Concept -
exp(coef)-(Hazard Ratio):
1. If this is < 1, the factor reduces the risk of churn (it's a good thing).
2. If this is > 1, the factor increases the risk of churn (it's a bad thing).

p (p-value):

1. If this is < 0.05, the factor's effect is statistically significant and real.

2. If this is > 0.05, the effect is likely due to random chance and not significant.

Total_Orders: The exp(coef) is 0.63 (which is less than 1) and the p-value is <0.005.
Insight: This is our most important finding. It means that for every additional order a customer places, their risk of churning decreases by about 37% (1.00 - 0.63 = 0.37). This is a highly significant factor that drives customer retention.

Avg_Discount & Total_Sales: For both of these factors, the p-value is very high (0.92 and 0.89).
Insight: This tells us that, surprisingly, the total amount a customer spends and the average discount they receive have no statistically significant effect on whether they churn or not.

In [ ]:
cph.plot()

* The vertical dashed line represents "no effect."
* Factors to the left of the line reduce churn risk.
* Factors to the right increase churn risk.

# **Bayesian A/B Testing for Discount Effectiveness**

Now let's answer the question - "How likely is it that offering a discount is better than offering no discount?"

In [ ]:
import pymc as pm
import arviz as az

In [ ]:
data = df[df['Quantity'] < 10].copy() #filtering data for simplicity

In [ ]:
# Group A (Control)
control_group = data[data['Discount'] == 0]['Quantity']
# Group B (Treatment)
treatment_group = data[data['Discount'] > 0]['Quantity']

In [ ]:
print(len(control_group))#no discount
print(len(treatment_group))#with discount

In [ ]:
#now let's define bayesian model
with pm.Model() as model:
    avg_control = pm.Gamma('avg_control', alpha=1.0, beta=1.0)
    avg_treatment = pm.Gamma('avg_treatment', alpha=1.0, beta=1.0)
    control_likelihood = pm.Poisson('control_likelihood', mu=avg_control, observed=control_group)
    treatment_likelihood = pm.Poisson('treatment_likelihood', mu=avg_treatment, observed=treatment_group)
    diff_of_means = pm.Deterministic('diff_of_means', avg_treatment - avg_control)
    treatment_is_better = pm.Deterministic('treatment_is_better', diff_of_means > 0)
#running the simulation
with model:
    trace = pm.sample(2000, tune=1000, cores=1)


In [ ]:
#let's analyse and plot
az.plot_posterior(trace, var_names=['avg_control', 'avg_treatment', 'diff_of_means'])



* This analysis strongly indicates that offering a discount is not an effective strategy for increasing the number of items customers purchase per transaction.

# Market Share Velocity Calculation

The goal is to measure the rate of change of market share for our main product categories. A simple market share number tells you where you are now, but velocity tells you if you are gaining or losing ground and how quickly. This is a leading indicator of future performance.

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['YearQuarter'] = df['Order Date'].dt.to_period('Q').astype(str)
# total sales per quarter
total_quarterly_sales = df.groupby('YearQuarter')['Sales'].sum().reset_index()
total_quarterly_sales.rename(columns={'Sales': 'Total_Quarterly_Sales'}, inplace=True)
# sales per category per quarter
category_quarterly_sales = df.groupby(['YearQuarter', 'Category'])['Sales'].sum().reset_index()
#calculate market share
market_data = pd.merge(category_quarterly_sales, total_quarterly_sales, on='YearQuarter')
market_data['Market_Share'] = (market_data['Sales'] / market_data['Total_Quarterly_Sales']) * 100
# Calculate Market Share Velocity
market_data.sort_values(by=['Category', 'YearQuarter'], inplace=True)
# Calculate the change from the previous quarter
market_data['Market_Share_Velocity'] = market_data.groupby('Category')['Market_Share'].pct_change() * 100
# Show the velocity for the last few quarters for each category
print("--- Market Share Velocity (% Change from Previous Quarter) ---")
print(market_data.dropna().groupby('Category').tail(3))
avg_velocity = market_data.groupby('Category')['Market_Share_Velocity'].mean().reset_index()
print("\n--- Average Quarterly Market Share Velocity ---")
print(avg_velocity)

* The first thing we notice is how much the velocity fluctuates. For example, Furniture's market share grew by a massive 74.7% in Q2 but then fell by 14.8% in Q3. This indicates that market share is not stable and is highly dependent on sales within a specific quarter.
*  In the most recent quarter (2017Q4), both Furniture (+12.5%) and Technology (+8.3%) saw healthy growth in their market share, while Office Supplies (-17.5%) saw a significant decline.
* With an average quarterly growth of 5.7%, the Office Supplies category is consistently expanding its share of the business over the long term.The Furniture category is also growing steadily, with an average quarterly velocity of 3.8%.The Technology category has an average velocity of only 0.2%. This is a critical insight. While we know from previous analysis that Technology is the most profitable and has the highest market share, this result tells us that its dominant position is stable but not growing.

# **Product Portfolio Health Score**

In [ ]:
# Aggregate data by product
product_health_data = df.groupby('Product Name').agg(Total_Sales=('Sales', 'sum'), Total_Profit=('Profit', 'sum'), Avg_Discount=('Discount', 'mean')).reset_index()
# Filter out products with zero or negative sales to avoid division by zero
product_health_data = product_health_data[product_health_data['Total_Sales'] > 0]
product_health_data['Profit_Margin'] = product_health_data['Total_Profit'] / product_health_data['Total_Sales']
product_health_data['Sales_Score'] = product_health_data['Total_Sales'].rank(pct=True) # Sales Score: Higher sales = higher score
# Profit Score: Higher profit margin = higher score
product_health_data['Profit_Score'] = product_health_data['Profit_Margin'].rank(pct=True)
# Discount Score: Lower discount = higher score
product_health_data['Discount_Score'] = 1 - product_health_data['Avg_Discount'].rank(pct=True)

# Final Health Score
# We give equal weight to each of the three components
product_health_data['Health_Score'] = (product_health_data['Sales_Score'] + product_health_data['Profit_Score'] + product_health_data['Discount_Score']) / 3 * 100 # Scale to 100

print("--- Top 10 Healthiest Products ---")
print(product_health_data.sort_values('Health_Score', ascending=False).head(10)[['Product Name', 'Health_Score', 'Total_Sales', 'Profit_Margin', 'Avg_Discount']])

print("\n--- Bottom 10 Unhealthiest Products ---")
print(product_health_data.sort_values('Health_Score', ascending=True).head(10)[['Product Name', 'Health_Score', 'Total_Sales', 'Profit_Margin', 'Avg_Discount']])


here health score was evaluated by giving equal weightage to sales volume, profit margin, and discount level.

# **Revenue Quality Score**

Goal is assess the health and sustainability of the revenue generated by each product category.

In [ ]:
# A sale is 'High Quality' if it was profitable AND had no discount.
df['is_high_quality'] = (df['Profit'] > 0) & (df['Discount'] == 0)
df['high_quality_revenue'] = df.apply(lambda row: row['Sales'] if row['is_high_quality'] else 0, axis=1)
# Calculate total sales for each category
total_sales_by_category = df.groupby('Category')['Sales'].sum()
# Calculate the high-quality revenue for each category
high_quality_sales_by_category = df.groupby('Category')['high_quality_revenue'].sum()
revenue_quality_df = pd.DataFrame({'Total_Revenue': total_sales_by_category, 'High_Quality_Revenue': high_quality_sales_by_category})
revenue_quality_df['Revenue_Quality_Score'] = (revenue_quality_df['High_Quality_Revenue'] / revenue_quality_df['Total_Revenue']) * 100

# Display the results ---
print("--- Revenue Quality Score by Category ---")
print(revenue_quality_df.sort_values('Revenue_Quality_Score', ascending=False))


result shows Office Supplies is the most sustainable category. The biggest strategic challenge is the Furniture category.

# **Statistical Hypothesis Generation**

# Hypothesis Test 1: Technology vs. Furniture Profit

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
# Creating two separate samples for the test
# Sample 1: Profit from all sales in the 'Technology' category
tech_profit = df[df['Category'] == 'Technology']['Profit']
# Sample 2: Profit from all sales in the 'Furniture' category
furniture_profit = df[df['Category'] == 'Furniture']['Profit']
# Perform the Independent Samples T-Test
t_statistic, p_value = ttest_ind(tech_profit, furniture_profit, equal_var=False) # equal_var=False because the variances of the two groups might not be equal

print("--- Hypothesis Test: Technology Profit vs. Furniture Profit ---")
print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05  # Standard significance level
if p_value < alpha:
    print("We reject the null hypothesis. The average profit for Technology is significantly different from Furniture.")
else:
    print("We fail to reject the null hypothesis. There is no statistically significant difference in average profit.")


T-statistic: 6.7320: This indicates that the average profit for Technology is indeed much higher than the average profit for Furniture.
P-value: 0.0000: p-value this low means that the difference we observed is not due to random chance.

# Hypothesis Test 2: First Class vs. Standard Class Sales

In [ ]:
# Sample 1: Sales from all orders with 'First Class' shipping
first_class_sales = df[df['Ship Mode'] == 'First Class']['Sales']
# Sample 2: Sales from all orders with 'Standard Class' shipping
standard_class_sales = df[df['Ship Mode'] == 'Standard Class']['Sales']
t_statistic, p_value = ttest_ind(first_class_sales, standard_class_sales, equal_var=False)

print("--- Hypothesis Test: First Class Sales vs. Standard Class Sales ---")
print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

# Conclusion based on the p-value
alpha = 0.05
if p_value < alpha:
    print("We reject the null hypothesis. The average sales value for First Class shipping is significantly different from Standard Class.")
else:
    print("We fail to reject the null hypothesis. There is no statistically significant difference in average sales value.")

observation - the customers who choose premium 'First Class' shipping do not spend more on average than customers who choose 'Standard Class' shipping.

### Shipping Mode and Sales Value

"Orders shipped via 'First Class' have a significantly higher average sales value than orders shipped via 'Standard Class'."

In [ ]:
# Sample 1: Sales from all orders with 'First Class' shipping
first_class_sales = df[df['Ship Mode'] == 'First Class']['Sales']

# Sample 2: Sales from all orders with 'Standard Class' shipping
standard_class_sales = df[df['Ship Mode'] == 'Standard Class']['Sales']

# Perform the Independent Samples T-Test
t_statistic, p_value = ttest_ind(first_class_sales, standard_class_sales, equal_var=False)
print("--- Hypothesis Test: First Class Sales vs. Standard Class Sales ---")
print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("We reject the null hypothesis. The average sales value for First Class shipping is significantly different from Standard Class.")
else:
    print("We fail to reject the null hypothesis. There is no statistically significant difference in average sales value.")

# Interactive Sales and Profit Dashboard

In [ ]:
pip install dash

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


# Initialize the Dash App
app = dash.Dash(__name__)

# now defining the App Layout
app.layout = html.Div(children=[
    html.H1(children='Superstore Analytics Dashboard', style={'textAlign': 'center'}),

    html.Div(children='''
        An interactive dashboard to explore Sales and Profit data.
    ''', style={'textAlign': 'center'}),

    # Dropdown menu to select the metric
    dcc.Dropdown(
        id='metric-selector',
        options=[
            {'label': 'Sales', 'value': 'Sales'},
            {'label': 'Profit', 'value': 'Profit'}
        ],
        value='Sales',  # Default value
        style={'width': '50%', 'margin': '20px auto'}
    ),

    # Graph components that will be updated by the callback
    dcc.Graph(id='category-bar-chart'),
    dcc.Graph(id='time-series-chart')
])

# Define the Callback Function(This function connects the dropdown to the graphs)
@app.callback(
    [Output('category-bar-chart', 'figure'),
     Output('time-series-chart', 'figure')],
    [Input('metric-selector', 'value')]
)
def update_graphs(selected_metric):
    # Bar Chart by Category
    bar_fig = px.bar(
        df.groupby('Category')[selected_metric].sum().reset_index(),
        x='Category',
        y=selected_metric,
        title=f'Total {selected_metric} by Product Category'
    )

    # Time Series Chart
    time_series_data = df.groupby(pd.Grouper(key='Order Date', freq='M'))[selected_metric].sum().reset_index()
    line_fig = px.line(
        time_series_data,
        x='Order Date',
        y=selected_metric,
        title=f'Total {selected_metric} Over Time'
    )

    return bar_fig, line_fig

if __name__ == '__main__':
    app.run(debug=True, port=8051)

In [ ]:
import pandas as pd
import plotly.express as px
df['Order Date'] = pd.to_datetime(df['Order Date'])
# Create a 'Month-Year' column for grouping
df['MonthYear'] = df['Order Date'].dt.to_period('M').astype(str)
# Group by month and category to get monthly sales
monthly_sales = df.groupby(['MonthYear', 'Category'])['Sales'].sum().reset_index()
# Sort by date to ensure correct cumulative calculation
monthly_sales.sort_values(by='MonthYear', inplace=True)
# Calculate the cumulative sales for each category
monthly_sales['Cumulative_Sales'] = monthly_sales.groupby('Category')['Sales'].cumsum()
fig = px.bar(
    monthly_sales,
    x='Category',
    y='Cumulative_Sales',
    color='Category',
    animation_frame='MonthYear', # Creates a frame for each month
    animation_group='Category',  # Tracks each category across frames
    range_y=[0, monthly_sales['Cumulative_Sales'].max() * 1.1],
    title="Animated Bar Chart Race: Cumulative Sales by Category",
    labels={'Cumulative_Sales': 'Cumulative Sales', 'Category': 'Product Category'}
)
# Improve the layout and animation speed
fig.update_layout(
    transition={'duration': 200} # Speed of transition between frames in milliseconds
)

fig.show()


Inteteractive product co-purchase network

In [ ]:
import plotly.graph_objects as go
most_frequent_pairs = edge_counts.head(30)
G = nx.from_pandas_edgelist(most_frequent_pairs,
                             source='Product Pair',
                             target='Product Pair', # This is a trick to handle tuples in columns
                             edge_attr='Frequency')
G_corrected = nx.Graph()
for index, row in most_frequent_pairs.iterrows():
    product1, product2 = row['Product Pair']
    frequency = row['Frequency']
    G_corrected.add_edge(product1, product2, weight=frequency)

G = G_corrected
# Calculate node positions using a spring layout
pos = nx.spring_layout(G, k=0.8, iterations=50, seed=42)
# Create the edge trace
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')
# Create the node trace
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))
# Add node text (product names) and color based on connectivity
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(f'{adjacencies[0]}<br># of connections: {len(adjacencies[1])}')

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

# Create the Figure and Show It
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Interactive Network Graph of Product Co-Purchases',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="An interactive graph showing which products are most frequently bought together.",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()


CEO Executive Dashboard

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df['Order Date'] = pd.to_datetime(df['Order Date'])
df.sort_values('Order Date', inplace=True)
monthly_summary = df.resample('M', on='Order Date').agg(
    Monthly_Sales=('Sales', 'sum'),
    Monthly_Profit=('Profit', 'sum')
).reset_index()
# Calculate cumulative unique customers over time
df['cumulative_customers'] = df['Customer ID'].cumsum().astype(str).rank(method='dense').astype(int)
monthly_customers = df.groupby(pd.Grouper(key='Order Date', freq='M'))['cumulative_customers'].max().reset_index()
# using subplots to arrange the three charts
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"colspan": 2}, None],
           [{}, {}]],
    subplot_titles=("Total Sales Over Time", "Total Profit Over Time", "Cumulative Customer Growth")
)
#Chart 1: Total Sales Over Time
fig.add_trace(
    go.Scatter(x=monthly_summary['Order Date'], y=monthly_summary['Monthly_Sales'], mode='lines+markers', name='Sales'),
    row=1, col=1
)
# Chart 2: Total Profit Over Time
fig.add_trace(
    go.Scatter(x=monthly_summary['Order Date'], y=monthly_summary['Monthly_Profit'], mode='lines+markers', name='Profit'),
    row=2, col=1
)
# Chart 3: Cumulative Customer Growth
fig.add_trace(
    go.Scatter(x=monthly_customers['Order Date'], y=monthly_customers['cumulative_customers'], mode='lines', name='Customers', line=dict(color='green')),
    row=2, col=2
)
#Style and Finalize the Dashboard
fig.update_layout(
    title_text='Key Business Performance Indicators',
    height=600,
    showlegend=False
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df['Order Date'] = pd.to_datetime(df['Order Date'])

# Create a monthly summary for time-series analysis
monthly_financials = df.resample('M', on='Order Date').agg(
    Monthly_Sales=('Sales', 'sum'),
    Monthly_Profit=('Profit', 'sum')
).reset_index()
# Calculate monthly profit margin
monthly_financials['Profit_Margin'] = (monthly_financials['Monthly_Profit'] / monthly_financials['Monthly_Sales']) * 100
# Create a summary by sub-category
subcategory_profit = df.groupby('Sub-Category')['Profit'].sum().reset_index().sort_values('Profit', ascending=False)

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"colspan": 2}, None],
           [{}, {}]],
    subplot_titles=("Profit Margin Over Time (%)", "Sales vs. Profit Analysis", "Profit by Sub-Category")
)
fig.add_trace(
    go.Scatter(x=monthly_financials['Order Date'], y=monthly_financials['Profit_Margin'], mode='lines', name='Profit Margin'),
    row=1, col=1
)

# Sales vs. Profit
fig.add_trace(
    go.Scatter(x=monthly_financials['Order Date'], y=monthly_financials['Monthly_Sales'], name='Sales', line=dict(color='blue')),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=monthly_financials['Order Date'], y=monthly_financials['Monthly_Profit'], name='Profit', line=dict(color='red')),
    row=2, col=1
)

# Profit by Sub-Category
fig.add_trace(
    go.Bar(x=subcategory_profit['Sub-Category'], y=subcategory_profit['Profit'], name='Sub-Category Profit'),
    row=2, col=2
)


#  Style and Finalize the Dashboard
fig.update_layout(
    title_text='Financial Performance & Profitability Analysis',
    height=700,
    showlegend=True
)
# Make the sub-category labels easier to read
fig.update_xaxes(tickangle=45, row=2, col=2)

fig.show()


In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['MonthYear'] = df['Order Date'].dt.to_period('M')
# Find the most recent month in the dataset
last_month = df['MonthYear'].max()
previous_month = last_month - 1
# Filter data for the last month
last_month_data = df[df['MonthYear'] == last_month]
# Filter data for the previous month
previous_month_data = df[df['MonthYear'] == previous_month]

last_month_sales = last_month_data['Sales'].sum()
previous_month_sales = previous_month_data['Sales'].sum()
last_month_profit = last_month_data['Profit'].sum()
previous_month_profit = previous_month_data['Profit'].sum()

# Calculate sales growth
sales_growth = ((last_month_sales - previous_month_sales) / previous_month_sales) * 100

# Best and Worst Sub-Category in the Last Month
subcat_performance = last_month_data.groupby('Sub-Category')['Profit'].sum()
best_subcat = subcat_performance.idxmax()
best_subcat_profit = subcat_performance.max()

worst_subcat = subcat_performance.idxmin()
worst_subcat_profit = subcat_performance.min()

# Generate the Natural Language Summary
report = f"""
Report for Month: {last_month}
1.  Overall Performance:
    * Total Sales: ${last_month_sales:,.2f}
    * Total Profit: ${last_month_profit:,.2f}
    * Sales Growth (vs. prior month): {sales_growth:.2f}%

2.  Key Insights for the Month:
    * The best-performing product line was **'{best_subcat}'**, which generated **${best_subcat_profit:,.2f}** in profit.
    * The worst-performing product line was **'{worst_subcat}'**, which lost **${worst_subcat_profit:,.2f}**.
"""

print(report)


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error
import plotly.graph_objects as go
# Prophet requires the columns to be named 'ds' (datestamp) and 'y' (value).
df['Order Date'] = pd.to_datetime(df['Order Date'])
monthly_sales = df.resample('M', on='Order Date')['Sales'].sum().reset_index()
monthly_sales.rename(columns={'Order Date': 'ds', 'Sales': 'y'}, inplace=True)
# We will train the model on data up to the end of 2016 and test its accuracy on 2017 data.
train = monthly_sales[monthly_sales['ds'] < '2017-01-01']
test = monthly_sales[monthly_sales['ds'] >= '2017-01-01']
# We enable yearly seasonality as we know there are strong seasonal patterns in the data.
prophet_model = Prophet(yearly_seasonality=True, daily_seasonality=False)
prophet_model.fit(train)
# Create a dataframe for future dates (all 12 months of 2017)
future_dates = prophet_model.make_future_dataframe(periods=12, freq='M')
# Generate the forecast
prophet_forecast = prophet_model.predict(future_dates)
# Isolate the predictions for the test period (2017)
prophet_pred = prophet_forecast[prophet_forecast['ds'] >= '2017-01-01']['yhat']
# Calculate the Mean Absolute Percentage Error (MAPE)
mape_prophet = mean_absolute_percentage_error(test['y'], prophet_pred) * 100
print("Prophet Model Performance (MAPE)")
print(f"Prophet Model MAPE for 2017 Forecast: {mape_prophet:.2f}%")
print("\nMAPE indicates the average percentage difference between the forecast and the actual sales.")

# Use Prophet's built-in plotting function for a detailed view
fig1 = prophet_model.plot(prophet_forecast)
fig1.suptitle("Prophet Forecast Components", y=1.02)

# Create a cleaner plot for comparison
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=train['ds'], y=train['y'], mode='lines', name='Training Data'))
fig2.add_trace(go.Scatter(x=test['ds'], y=test['y'], mode='lines', name='Actual Sales (2017)', line=dict(color='black', width=3)))
fig2.add_trace(go.Scatter(x=test['ds'], y=prophet_pred, mode='lines', name='Prophet Forecast', line=dict(color='red', width=3, dash='dot')))

fig2.update_layout(title='Prophet Forecast vs. Actual Sales for 2017',
                   xaxis_title='Date',
                   yaxis_title='Sales')
fig2.show()


# Forecast Scenario Planning

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
monthly_sales = df.resample('M', on='Order Date')['Sales'].sum().reset_index()
monthly_sales.rename(columns={'Order Date': 'ds', 'Sales': 'y'}, inplace=True)

prophet_model = Prophet(yearly_seasonality=True, daily_seasonality=False)
prophet_model.fit(monthly_sales) # Fit on all data for a future forecast
future_dates = prophet_model.make_future_dataframe(periods=12, freq='M')
prophet_forecast = prophet_model.predict(future_dates)

# The forecast dataframe already contains the different scenarios.
# 'yhat' is the most-likely forecast.
# 'yhat_lower' is the pessimistic (worst-case) scenario.
# 'yhat_upper' is the optimistic (best-case) scenario.
forecast_period = prophet_forecast[prophet_forecast['ds'] > monthly_sales['ds'].max()]
fig = go.Figure()
# Plot historical data
fig.add_trace(go.Scatter(x=monthly_sales['ds'], y=monthly_sales['y'], mode='lines', name='Historical Sales'))
# Plot Most-Likely Forecast
fig.add_trace(go.Scatter(x=forecast_period['ds'], y=forecast_period['yhat'], mode='lines', name='Most-Likely Forecast', line=dict(color='blue', width=3)))
# Plot Best-Case Scenario
fig.add_trace(go.Scatter(x=forecast_period['ds'], y=forecast_period['yhat_upper'], mode='lines', name='Best-Case Scenario', line=dict(color='green', dash='dot')))
# Plot Worst-Case Scenario
fig.add_trace(go.Scatter(x=forecast_period['ds'], y=forecast_period['yhat_lower'], mode='lines', name='Worst-Case Scenario', line=dict(color='red', dash='dot')))
fig.update_layout(
    title='Sales Forecast Scenario Planning for the Next 12 Months',
    xaxis_title='Date',
    yaxis_title='Monthly Sales',
    legend_title='Scenario'
)
fig.show()

# Market Intelligence Forecasting

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
monthly_sales = df.resample('M', on='Order Date')['Sales'].sum().reset_index()
monthly_sales.rename(columns={'Order Date': 'ds', 'Sales': 'y'}, inplace=True)
# External Data: US Quarterly GDP Growth Rate (%)
# For this project, this data is provided. In a real-world scenario, you would source this from an economic data provider.
gdp_data = {
    'date': pd.to_datetime(['2014-01-01', '2014-04-01', '2014-07-01', '2014-10-01',
                           '2015-01-01', '2015-04-01', '2015-07-01', '2015-10-01',
                           '2016-01-01', '2016-04-01', '2016-07-01', '2016-10-01',
                           '2017-01-01', '2017-04-01', '2017-07-01', '2017-10-01',
                           '2018-01-01', '2018-04-01', '2018-07-01', '2018-10-01']),
    'gdp_growth': [-1.2, 4.5, 5.1, 2.1, 3.2, 3.4, 2.0, 1.1, 1.5, 2.0, 2.7, 2.4, 2.0, 2.2, 2.9, 3.5, 2.2, 3.5, 2.9, 1.1]
}
gdp_df = pd.DataFrame(gdp_data)
gdp_df['ds'] = gdp_df['date']
# using forward-fill merge algorithm  to apply the quarterly GDP data to each month within that quarter.
merged_df = pd.merge_asof(monthly_sales.sort_values('ds'),gdp_df.sort_values('ds'),on='ds',direction='backward')
# Build and Train the Multivariate Prophet Model
# Initialize the model
model_with_gdp = Prophet(yearly_seasonality=True, daily_seasonality=False)
# Add the external regressor
model_with_gdp.add_regressor('gdp_growth')
# Fit the model on the combined data
model_with_gdp.fit(merged_df)
# To forecast we need future values for our regressor. We'll assume a stable growth rate for the next year
future = model_with_gdp.make_future_dataframe(periods=12, freq='M')
#adding the gdp_growth column to our future dataframe --  this is for future analysis too
future_gdp = pd.merge_asof(future.sort_values('ds'),gdp_df.sort_values('ds'),on='ds',direction='backward')
# For dates beyond our known GDP data, we'll assume a stable 2.5% growth
future_gdp['gdp_growth'].fillna(2.5, inplace=True)
forecast_with_gdp = model_with_gdp.predict(future_gdp)
fig = model_with_gdp.plot(forecast_with_gdp)
fig.suptitle("Sales Forecast with GDP Growth as an External Factor", y=1.02)
# Plot the model components, including the new GDP component
fig_components = model_with_gdp.plot_components(forecast_with_gdp)


# Strategic Recommendations Engine

In [ ]:
def generate_recommendations(df):
    """
    Analyzes the superstore dataframe to generate a prioritized list of strategic recommendations.
    """
    recommendations = []

    # opportunity : Address Unprofitable Sub-Categories
    subcat_profit = df.groupby('Sub-Category')['Profit'].sum()
    unprofitable_subcats = subcat_profit[subcat_profit < 0]
    for subcat, loss in unprofitable_subcats.items():
        recommendations.append({
            'Opportunity': f"Address Unprofitable Sub-Category: {subcat}",
            'Impact': abs(loss),  # Impact is the total loss that can be saved
            'Effort': 6,  # Medium-High effort: requires investigation, pricing changes, etc.
            'Recommendation': f"The '{subcat}' sub-category lost ${abs(loss):,.2f}. Investigate cost structure, pricing, and discount strategy. Consider discontinuing if profitability cannot be improved."
        })

    #  Opportunity : Expand in Untapped Markets
    customer_per_state = df.groupby('State')['Customer ID'].nunique()
    untapped_states = customer_per_state[customer_per_state < 10]
    avg_sales_per_customer = df['Sales'].sum() / df['Customer ID'].nunique()
    for state, num_customers in untapped_states.items():
        potential_impact = avg_sales_per_customer * 50 # Estimate impact of acquiring 50 new customers
        recommendations.append({
            'Opportunity': f"Expand in Untapped Market: {state}",
            'Impact': potential_impact,
            'Effort': 8,  # High effort: requires marketing campaigns, logistics
            'Recommendation': f"The state of '{state}' has only {num_customers} customers. Launch a targeted digital marketing campaign to increase market penetration and acquire new customers."
        })

    # Opportunity 3: Re-engage High-Value, At-Risk Customers
    # (Using a simplified RFM logic for this engine)
    snapshot_date = df['Order Date'].max() + pd.Timedelta(days=1)
    rfm = df.groupby('Customer ID').agg({
        'Order Date': lambda date: (snapshot_date - date.max()).days,
        'Sales': 'sum'
    })
    rfm.rename(columns={'Order Date': 'Recency', 'Sales': 'MonetaryValue'}, inplace=True)

    # Defining  'At-Risk': High value (top 25%) but not recent (recency > 180 days)
    at_risk_customers = rfm[(rfm['MonetaryValue'] > rfm['MonetaryValue'].quantile(0.75)) & (rfm['Recency'] > 180)]
    if not at_risk_customers.empty:
        at_risk_impact = at_risk_customers['MonetaryValue'].sum()
        recommendations.append({
            'Opportunity': "Re-engage High-Value, At-Risk Customers",
            'Impact': at_risk_impact,
            'Effort': 3,
            'Recommendation': f"There are {len(at_risk_customers)} high-value customers who have not purchased in over 6 months. Recommendation is to Launch a targeted re-engagement campaign with a special offer to win them back."
        })#giving suggestion for regaining the lost customers
    #  Scoring and Ranking
    rec_df = pd.DataFrame(recommendations)
    rec_df['Impact_Score'] = rec_df['Impact'].rank(pct=True)
    rec_df['Effort_Score'] = rec_df['Effort'].rank(pct=True)
    rec_df['Priority_Score'] = rec_df['Impact_Score'] / rec_df['Effort_Score']
    # Sort by the highest priority
    rec_df.sort_values('Priority_Score', ascending=False, inplace=True)
    return rec_df

top_recommendations = generate_recommendations(df)
for index, row in top_recommendations.head(5).iterrows():
    print(f"\nOPPORTUNITY: {row['Opportunity']}")
    print(f"  - PRIORITY SCORE: {row['Priority_Score']:.2f}")
    print(f"  - RECOMMENDATION: {row['Recommendation']}")
